In [1]:
# data handling and analysis libraries
import geopandas as gpd
import pandas as pd
import numpy as np

# machine learning
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

load our main GeoDataFrame

In [2]:
gdf = gpd.read_file('cells_finish.shp')